# Загрузка Pandas и очистка данных

In [ ]:
import pandas as pd
import ast
import geonamescache
import requests
from bs4 import BeautifulSoup

In [ ]:
df = pd.read_csv('main_task.csv')


In [ ]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [ ]:
# Исправляем ошибки в названиях городов для последующего извлечение дополнительной информации из внешних источников
for item in [{"Krakow": "Kraków"}, {"Oporto": "Porto"}, {"Zurich": "Zürich"}]:
    df.City.replace(item, inplace=True)

# Заполняем пустые значения колонки 'Number of Reviews' и приводим количество отзывов к целочисленному виду
df["Number of Reviews"] = df["Number of Reviews"].fillna(0)
df["Number of Reviews"] = df["Number of Reviews"].apply(lambda x: int(x))

# Приводим цены к числовому виду

transformed_prices = {"$": 1, "$$ - $$$": 2, "$$$$": 3}
df['Price Range'] = df['Price Range'].apply(lambda x: transformed_prices[x] if type(x) == str else 0)
#______________________________________________________________________________________________________
# Колонка 'Cuisine Style':
## приводим к листу тех значений, тип которых - строка
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: ast.literal_eval(x) if type(x) == str else x)

## заполняем пропуски
df['Cuisine Style'] = df['Cuisine Style'].fillna(1)

## Cоздаем колонку с количеством кухонь для каждого ресторана
df['Cuisines_amount'] = df['Cuisine Style'].apply(lambda x: len(x) if type(x) == list else x)

## Создаем dummy переменные для каждой отдельной кухни и ее наличии или отсутсвии в каждом ресторане
all_cuisines = list()
df['Cuisine Style'].apply(lambda x: all_cuisines.extend(x) if type(x) == list else x)

for cuisine in set(all_cuisines):
    df[cuisine] = df['Cuisine Style'].apply(lambda x: 1 if type(x) == list and cuisine in x else 0)

#__________________________________________________________________________________________________________

# Колонка 'Reviews':

## Заменяем пустые значения в колонке
df.Reviews = df.Reviews.apply(lambda x: ast.literal_eval(x.replace('nan', '"average"')))

## Разделяем колонку Reviews на две отдельных колонки: отзывы и даты отзывов
df.Reviews, df["Reviews_dates"] = df.Reviews.str

## приводим дату отзывов к datetime формату
df.Reviews_dates = df.Reviews_dates.apply(lambda x: pd.to_datetime(x))

## Находим разницу в днях между датами двух последних отзывов
df['feedback_period'] = df.Reviews_dates.apply(lambda x: max(x) - min(x) if len(x) == 2 else 0)

## Приводим полученный результат к целочисленному типу
df['feedback_period'] = df['feedback_period'].apply(lambda x: x.days if type(x) != int else x)


definitions = {
    "3": ["excellent", "best", "wonderful", "unique", "great", "i love", "fab food", "amazing", "gem", "eccellente",
          "fantastic", "loving", "beautifull", "amaze", "epic", "delicious", "perfect"],
    "2": ["good", "nice", "reasonable", "pretty", "friendly", "cute", "attractive"],
    "1": ["average", "not exceptional", "overwhelmed"],
    "-1": ["disappoint", "boring", "disgusting", "terrible", "unfriendly", "awful"]}


# Проходим каждый отзыв в ресторане и ищем наличие слова из словаря definitions. Если нашли увеличиваем счетчик на значение ключа из
# definitions. Если ни одно слово из отзыва не встречается в definitions , значит в ресторане ничего
#  особенного нет, и воспринимается значение как 0

def reviews_filter(reviews_list):
    counter = 0
    if type(reviews_list) == list and reviews_list:
        for review in reviews_list:
            for key in definitions:
                for word in definitions[key]:
                    if word in review.lower():
                        counter += int(key)

    return counter

#  Создаем колонку содержащую рейтинг ресторана на основе отзывов
df["reviews_rate"] = df.Reviews.apply(reviews_filter)

#__________________________________________________________________________________________________________

# создаем колонку количества населения городов из колокни City (Нужна дополнительная библиотека geonamescache)

## Создаем перечень  всех городов из колонки City
all_cities = set(df.City.values)


# Получаем информацию о каждом городе по его названию в виде списка.  В случае когда названий городов несколько в списке
# например Athens 4 значение 3 из которых находятся в США, то берем значение первого элемента в списке, т.к. первый элемент
# является известным городом. Наприме как в пример выше, то это будет столица Греции
gc = geonamescache.GeonamesCache()
prepared_info = dict()

for city in all_cities:
    cities = gc.get_cities_by_name(city)
    required_city = cities[0]
    city_key = list(required_city.keys())[0]
    prepared_info[city] = required_city[city_key]["population"]

# создаем колонку city_population c населением городов
df['city_population'] = df.City.apply(lambda x: prepared_info[x])

# создаем  dummy переменную городов и помечем в ней присутствие или отсутсвие каждого ресторана
df = pd.get_dummies(df, columns=['City'])

#__________________________________________________________________________________________________________

# Находим количество туристов на википедии для каждого города и создаем колонку  с их количеством
#  данные проверяем за 2018-й год, если города в списке нет - штош, значит город неочень популярен для туристов и
#  значение туристов в нем будет 0
resp = requests.get("https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors")
soup = BeautifulSoup(resp.content, 'html.parser')

# Извлекаем таблицу рейтинга городов и количеству туристов
table = soup.find("table", {"class": "wikitable sortable"})
tr = table.find_all("tr")

all_tourists = dict()

#  Обходим таблицу по названию города
for it in tr[1:]:

    city = it.find_all("td")[2].text.strip()
    if city in all_cities:

        # Берем данные из таблицы за 2018-й год
        tourists = "".join(it.find_all("td")[4].text.split(","))
        if len(tourists) == 1:

            # Если информации за 2018- год нет, то берем из той же таблицы следующую колонку - данные за 2016-й год
            all_tourists[city] = int("".join(it.find_all("td")[5].text.split(",")))
        else:
            all_tourists[city] = int(tourists)

# Собственно созлаем колонку с данными которые извлекли выше
df['tourists'] = df.City.apply(lambda x: all_tourists[x] if x in all_tourists.keys() else 0)

#__________________________________________________________________________________________________________

for column in df.columns:
    if df[column].dtype == object:
        df = df.drop(column, axis=1)
    else:
        df[column] = df[column].fillna(df[column].mean())

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))